In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import pprint


In [2]:
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def youtube_search(query):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_1050061667289-elvvcgoovn5phklinuh4bfclgn5dmjkt.apps.googleusercontent.com.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.search().list(
        part="snippet",
        q=query,
        maxResults=50
    )
    return request.execute()

In [19]:
import csv

with open('queries.csv', newline='') as f:
    reader = csv.reader(f)
    queries = list(reader)

print(queries)

[['farmer protests']]


In [20]:
responses = []
for query in queries:
    responses.append(youtube_search(query[0]))
pprint.pprint(responses)
    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1050061667289-elvvcgoovn5phklinuh4bfclgn5dmjkt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=uNSPYcxN0ZDddQbePVl1sWbCooBxwi&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g5erfnLz3nbuD5c6xCBfETd5OdvkJmC1fgg7krJGN3f45Y0lmH61hE
[{'etag': 'ITiRm_4Qyg0Hp9r733f-lRplAPM',
  'items': [{'etag': 'OfrioKA6qx9q76B5xEa7Su61UFo',
             'id': {'kind': 'youtube#video', 'videoId': 'gRW_O9KjqWY'},
             'kind': 'youtube#searchResult',
             'snippet': {'channelId': 'UC16niRr50-MSBwiO3YDb3RA',
                         'channelTitle': 'BBC News',
                         'description': 'For more than two and a half months, '
                                        'hundreds of thousands of farmers in '
                                  

In [21]:
videos = []

for response in responses:
    for item in response['items']:
        videos.append({
            "id":item['id']['videoId'],
            "channelId":item['snippet']['channelId'],
            "channelName":item['snippet']['channelTitle'],
            "videoTitle":item['snippet']['title'],
            "desc":item['snippet']['description']
        })
df = pd.DataFrame(videos, columns=['channelId','channelName','desc','id','videoTitle'])
df.to_csv('results.csv')

In [23]:
from __future__ import unicode_literals
import youtube_dl
import os

os.system('mkdir downloads')
os.system('cd downloads')

#to download the videos
ydl = youtube_dl.YoutubeDL({'outtmpl': './downloads/%(title)s.%(ext)s'})

# for video in videos:
for video in videos:
    url = f"https://www.youtube.com/watch?v={video['id']}"
    with ydl:
        ydl.download([url])

[youtube] gRW_O9KjqWY: Downloading webpage
[download] Destination: ./downloads/Farmer protests - India's sedition law used to muffle dissent - BBC News.f137.mp4
[download] 100% of 59.78MiB in 00:0579MiB/s ETA 00:001
[download] Destination: ./downloads/Farmer protests - India's sedition law used to muffle dissent - BBC News.f140.m4a
[download] 100% of 3.30MiB in 00:0066MiB/s ETA 00:003
[ffmpeg] Merging formats into "./downloads/Farmer protests - India's sedition law used to muffle dissent - BBC News.mp4"
Deleting original file ./downloads/Farmer protests - India's sedition law used to muffle dissent - BBC News.f137.mp4 (pass -k to keep)
Deleting original file ./downloads/Farmer protests - India's sedition law used to muffle dissent - BBC News.f140.m4a (pass -k to keep)
